# CE Enablement Session - Practice 01
- ## Workflow: IMG --> Nano --> VEO

### 1. (권장) 가상환경 생성 및 활성화
python -m venv venv
#### macOS/Linux:
source venv/bin/activate

#### 2. 필수 파이썬 라이브러리 설치
pip install --upgrade google-cloud-aiplatform Pillow ipython

In [1]:
# ==============================================================================
# 1. 패키지 임포트 및 전역 설정
# ==============================================================================
import os
import json
import base64
import io
import mimetypes
from datetime import datetime
from typing import Dict, List, Optional
# Third-party libraries
import requests
from PIL import Image as PIL_Image

# Google Cloud and Generative AI libraries
from google import genai
from google.genai import types
from google.genai.types import (
    EditImageConfig,
    GenerateImagesConfig,
    Image,
    MaskReferenceConfig,
    MaskReferenceImage,
    RawReferenceImage,
)

# IPython display for notebooks
from IPython.display import display

# --- 전역 상수 정의 ---
# 'jc-gcp-project'를 당신의 GCP 프로젝트 ID로 변경해주세요.
PROJECT_ID = "jc-gcp-project"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")
OUTPUT_DIR = "veo_story_telling"

# --- 전역 변수 및 초기화 ---
# 스크립트 시작 시 출력 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# API 클라이언트를 스크립트 시작 시 한 번만 초기화하여 모든 함수에서 재사용합니다.
# 이렇게 하면 반복적인 인증 및 객체 생성 오버헤드가 사라집니다.
try:
    CLIENT = genai.Client(vertexai=True, project=PROJECT_ID, location="global")
    print("✅ GenAI 클라이언트가 성공적으로 초기화되었습니다.")
except Exception as e:
    print(f"❌ GenAI 클라이언트 초기화 실패: {e}")
    CLIENT = None

# VEO 비디오 생성을 위한 Image 데이터 클래스
# 함수 외부로 이동하여 반복적인 클래스 정의를 방지합니다.
class VeoImage:
    """VEO API에 이미지 데이터를 전달하기 위한 간단한 데이터 클래스입니다."""
    def __init__(self, gcs_uri=None, image_bytes=None, mime_type=None):
        self.gcs_uri = gcs_uri
        self.image_bytes = image_bytes
        self.mime_type = mime_type

    def __repr__(self):
        bytes_repr = str(self.image_bytes[:60]) + '...' if self.image_bytes else None
        return (f"VeoImage(gcs_uri={self.gcs_uri}, "
                f"mime_type='{self.mime_type}', "
                f"image_bytes={bytes_repr})")

✅ GenAI 클라이언트가 성공적으로 초기화되었습니다.


In [2]:
# ==============================================================================
# 2. 헬퍼 함수 (Helper Functions)
# ==============================================================================

def _load_image_part(path: str) -> Optional[types.Part]:
    """
    로컬 경로 또는 GCS URI에서 이미지를 로드하여 genai.types.Part 객체로 반환합니다.
    이 함수는 코드 중복을 방지하기 위해 사용됩니다.
    """
    if path.startswith("gs://"):
        mime_type, _ = mimetypes.guess_type(path)
        if not mime_type:
            mime_type = "image/jpeg"  # GCS 기본값
        print(f"✅ GCS 이미지 로드: {path}")
        return types.Part.from_uri(file_uri=path, mime_type=mime_type)
    else:
        try:
            mime_type, _ = mimetypes.guess_type(path)
            if not mime_type:
                raise IOError(f"파일의 MIME 타입을 확인할 수 없습니다: {path}")
            
            with open(path, "rb") as f:
                image_bytes = f.read()
            
            print(f"✅ 로컬 이미지 로드: {path}")
            return types.Part.from_bytes(data=image_bytes, mime_type=mime_type)
        except FileNotFoundError:
            print(f"❌ 오류: 로컬 파일 '{path}'를 찾을 수 없습니다.")
        except Exception as e:
            print(f"❌ 오류: 로컬 파일 '{path}' 처리 중 문제 발생: {type(e).__name__} - {e}")
    return None

In [3]:
# ==============================================================================
# 3. 핵심 생성 함수 (Core Generative Functions)
# ==============================================================================

def generate_prompt_from_images(user_prompt: str, image_paths: List[str]) -> Optional[str]:
    """
    사용자 프롬프트와 이미지를 기반으로 광고에 최적화된 새로운 프롬프트를 생성합니다.
    """
    if not CLIENT:
        print("오류: GenAI 클라이언트가 초기화되지 않았습니다.")
        return None
    if not image_paths:
        print("오류: 하나 이상의 이미지 파일 경로를 제공해야 합니다.")
        return None
    if len(image_paths) > 2:
        print("경고: 최대 2개의 이미지만 처리됩니다. 처음 2개의 이미지를 사용합니다.")
        image_paths = image_paths[:2]

    try:
        default_prompt = """
        You will receive a photo and a user prompt. Your task is to analyze both and generate a detailed and specific prompt that enhances the quality and relevance of the advertisement, **while explicitly prioritizing the preservation of existing subjects and objects in the original image.**

        For example, if the user asks to composite a person into a photo, analyze the photo to suggest appropriate poses for the person that *integrate seamlessly with the existing scene*. If the user asks to change the text on a poster, create a prompt that ensures the new text blends seamlessly with the original design *without altering the original visual elements unnecessarily*.

        The goal is to enhance the advertisement by subtly modifying or adding elements that complement the original, rather than replacing or heavily altering it.

        Example Output:"Enhance the existing image by subtly adjusting the lighting to highlight the cosmetic product, ensuring the woman's pose and expression remain the same but are visually optimized for the product's branding and target audience. Do not alter the woman's appearance or the product's original form."        
       
        The customer request is : 
        """
        parts = [types.Part.from_text(text=default_prompt), types.Part.from_text(text=user_prompt)]

        for path in image_paths:
            image_part = _load_image_part(path)
            if image_part:
                parts.append(image_part)

        system_instruction_text = """
        You are a prompt expert specializing in advertising. Your task is to take a photo and a user prompt as input and generate a refined prompt that is optimized for creating effective advertisements.
        """

        model = "gemini-2.5-flash"
        contents = [types.Content(role="user", parts=parts)]

        generate_content_config = types.GenerateContentConfig(
            temperature=1, top_p=1, seed=0, max_output_tokens=8192,
            safety_settings=[
                types.SafetySetting(category=c, threshold="BLOCK_NONE") for c in
                ["HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_DANGEROUS_CONTENT", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_HARASSMENT"]
            ],
            response_mime_type="application/json",
            response_schema={"type": "OBJECT", "properties": {"response": {"type": "STRING"}}},
            system_instruction=[types.Part.from_text(text=system_instruction_text)],
        )
        
        print("\n생성 중...")
        response = CLIENT.models.generate_content(
            model=model, contents=contents, config=generate_content_config
        )
        print("생성 완료.")
        
        try:
            parsed_json = json.loads(response.text)
            return parsed_json.get('response')
        except json.JSONDecodeError:
            print(f"❌ 오류: API 응답을 JSON으로 파싱할 수 없습니다. 원본 응답: {response.text}")
            return None

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
        return None


def generate_edited_image(prompt: str, image_paths: List[str], save_image: bool = True, output_path: Optional[str] = None) -> tuple[Optional[bytes], Optional[str]]:
    """
    이미지를 생성하고, 텍스트만 반환될 경우 지능적으로 재시도하여 결과물을 파일로 저장합니다.
    """
    if not CLIENT:
        print("오류: GenAI 클라이언트가 초기화되지 않았습니다.")
        return None, None

    user_parts = []
    for path in image_paths:
        image_part = _load_image_part(path)
        if image_part:
            user_parts.append(image_part)
    
    user_parts.append(types.Part.from_text(text=prompt))
    initial_contents = [types.Content(role="user", parts=user_parts)]

    generate_content_config = types.GenerateContentConfig(
        temperature=1, top_p=0.95, max_output_tokens=32768,
        response_modalities=["TEXT", "IMAGE"],
        safety_settings=[
            types.SafetySetting(category=c, threshold="BLOCK_NONE") for c in
            ["HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_DANGEROUS_CONTENT", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_HARASSMENT"]
        ],
    )

    image_found = False
    model_response_text = ""
    received_image_bytes = b""
    final_path = None

    def process_stream_response(current_contents):
        nonlocal image_found, model_response_text, received_image_bytes, final_path
        
        for chunk in CLIENT.models.generate_content_stream(
            model="gemini-2.5-flash-image-preview", contents=current_contents, config=generate_content_config,
        ):
            for part in chunk.candidates[0].content.parts:
                if part.text:
                    model_response_text += part.text
                    print(part.text, end="")
                
                if part.inline_data:
                    image_found = True
                    received_image_bytes = part.inline_data.data
                    print("\n\n--- ✅ 이미지 수신 완료 ---")
                    
                    if save_image:
                        if output_path:
                            final_path = output_path
                        else:
                            default_dir = os.path.join(OUTPUT_DIR, "sample_imgs")
                            os.makedirs(default_dir, exist_ok=True)
                            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                            filename = f"output_{timestamp}.png"
                            final_path = os.path.join(default_dir, filename)
                        
                        try:
                            with open(final_path, "wb") as f:
                                f.write(received_image_bytes)
                            print(f"💾 이미지가 '{os.path.abspath(final_path)}' 파일로 저장되었습니다.")
                        except IOError as e:
                            print(f"❌ 파일 저장 중 오류 발생: {e}")
    
    print(f"--- 1차 이미지 생성 시도 (입력 이미지 {len(image_paths)}개) ---")
    process_stream_response(initial_contents)
    
    if not image_found:
        print("\n\n--- ⚠️ 1차 시도에서 이미지가 생성되지 않았습니다. 2차 시도를 시작합니다. ---")
        
        retry_contents = list(initial_contents)
        retry_contents.append(types.Content(role="model", parts=[types.Part.from_text(text=model_response_text)]))
        retry_contents.append(types.Content(role="user", parts=[types.Part.from_text(text="네, 좋습니다. 어떤 방법을 추가해서라도 이미지를 생성해 주세요.")]))

        image_found = False
        model_response_text = ""
        received_image_bytes = b""

        process_stream_response(retry_contents)
        print("\n\n--- ✅ 2차 시도 완료 ---")

    if not image_found:
        print("\n\n--- ❌ 최종적으로 이미지 생성에 실패했습니다. ---")
    
    return received_image_bytes, final_path


def generate_video_from_image(image_path: str, scene_story: str, scene_idx: int) -> str:
    """
    단일 이미지와 스토리 프롬프트를 기반으로 비디오를 생성합니다.
    """
    import time
    if not CLIENT:
        print("오류: GenAI 클라이언트가 초기화되지 않았습니다.")
        return ""

    try:
        mime_type, _ = mimetypes.guess_type(image_path)
        if not mime_type:
            raise ValueError(f"Could not determine MIME type for {image_path}")

        with open(image_path, "rb") as image_file:
            image_byte_data = image_file.read()

        image_instance = VeoImage(
            image_bytes=image_byte_data,
            mime_type=mime_type
        )
        print("✅ VeoImage 객체가 성공적으로 생성되었습니다!")

    except FileNotFoundError:
        print(f"❌ 오류: 파일을 찾을 수 없습니다 '{image_path}'")
        return ""
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return ""
    
    prompt = f"""Generate a video from this image that shows: {scene_story}
    The video should be smooth and cinematic, lasting 8 seconds.
    Create natural movement and progression from the starting image."""
    
    try:
        operation = CLIENT.models.generate_videos(
            model="veo-3.0-generate-preview",
            prompt=prompt,
            image=image_instance,
        )

        while not operation.done:
            print("비디오 생성 완료를 기다리는 중...")
            time.sleep(10)
            operation = CLIENT.operations.get(operation)

        if operation.done and not operation.error:
            video_bytes = operation.response.generated_videos[0].video.video_bytes
            video_path = f"{OUTPUT_DIR}/scene_{scene_idx}_video.mp4"
            with open(video_path, 'wb') as f:
                f.write(video_bytes)
            print(f"'{video_path}' 파일로 비디오를 성공적으로 저장했습니다.")
            return video_path
        else:
            print("비디오 생성 작업이 성공적으로 완료되지 않았습니다.")
            if operation.error:
                print(f"오류: {operation.error}")
            return ""

    except Exception as e:
        print(f"❌ 비디오 생성 중 오류 발생: {e}")
        return ""



In [4]:
# ==============================================================================
# 4. 이미지 편집/아웃페인팅 함수
# ==============================================================================

def get_bytes_from_pil(image: PIL_Image.Image) -> bytes:
    """PIL 이미지를 PNG 바이트로 변환합니다."""
    byte_io_png = io.BytesIO()
    image.save(byte_io_png, "PNG")
    return byte_io_png.getvalue()

def pad_to_target_size(source_image, target_size=(1536, 1536), mode="RGB", vertical_offset_ratio=0, horizontal_offset_ratio=0, fill_val=255):
    """아웃페인팅을 위해 이미지를 특정 크기로 패딩합니다."""
    orig_w, orig_h = source_image.size
    target_w, target_h = target_size
    
    insert_x = int((target_w - orig_w) / 2 + horizontal_offset_ratio * target_w)
    insert_y = int((target_h - orig_h) / 2 + vertical_offset_ratio * target_h)
    insert_x = min(max(0, insert_x), target_w - orig_w)
    insert_y = min(max(0, insert_y), target_h - orig_h)
    
    color = (fill_val, fill_val, fill_val) if mode == "RGB" else fill_val
    padded_image = PIL_Image.new(mode, target_size, color=color)
    padded_image.paste(source_image, (insert_x, insert_y))
    return padded_image

def pad_image_and_mask(image_pil, mask_pil, target_size, v_offset, h_offset):
    """이미지와 마스크를 대상 크기에 맞게 조정하고 패딩합니다."""
    image_pil.thumbnail(target_size)
    mask_pil.thumbnail(target_size)
    
    image_pil = pad_to_target_size(image_pil, target_size, "RGB", v_offset, h_offset, 0)
    mask_pil = pad_to_target_size(mask_pil, target_size, "L", v_offset, h_offset, 255)
    return image_pil, mask_pil

In [5]:
# ==============================================================================
# 5. 메인 실행 로직
# ==============================================================================

In [6]:
# --- 1단계: 프롬프트 생성 ---
print("--- 🚀 1단계: 광고 카피용 프롬프트 생성 ---")
prompt_text = "광고 이미지 속 한글을 잘 어울리는 영어로 변경하고 싶어."
image_files = ["veo_story_telling/sample_imgs/two_people_ads.png"]
# 생성된 프롬프트를 변수에 저장
generated_prompt = generate_prompt_from_images(user_prompt=prompt_text, image_paths=image_files)
print(generated_prompt)

--- 🚀 1단계: 광고 카피용 프롬프트 생성 ---
✅ 로컬 이미지 로드: veo_story_telling/sample_imgs/two_people_ads.png

생성 중...
생성 완료.
Identify the Korean text "부드러움과 감각적인 향 어노브로 완성하는 대담한 우아함" in the image. Replace this specific Korean text with the English phrase "A bold elegance perfected with soft and sensual ANOVE fragrance.". Ensure that all other elements of the original image, including the models, background, lighting, and the existing English text "UNLEASH YOUR" and "Bold Elegance", remain completely unchanged. The new English text should seamlessly adopt the font style, color, size, and spatial alignment of the original Korean text to maintain visual consistency.


In [7]:
# # --- 1단계: 프롬프트 생성 ---
# print("--- 🚀 1단계: 광고 카피용 프롬프트 생성 ---")
# prompt_text = "첫번째 사진 속 모델이 두번째 제품을 광고하는 사진으로 만들고 싶어. 최대한 자연스럽게 제품 사용하는 것처럼 만들고, 이미지속 글자는지워"
# image_files = ["veo_story_telling/sample_imgs/rose.png", "veo_story_telling/sample_imgs/product_02_cosme.png"]
# # 생성된 프롬프트를 변수에 저장
# generated_prompt = generate_prompt_from_images(user_prompt=prompt_text, image_paths=image_files)
# print(generated_prompt)

In [ ]:
# --- 2단계: 이미지 편집 ---
print("\n--- 🚀 2단계: 생성된 프롬프트를 사용하여 이미지 편집 ---")
edited_image_bytes, edited_image_path = generate_edited_image(
    prompt=generated_prompt,
    image_paths=image_files
)

print("\n--- ✅ 편집된 이미지 ---")
display(PIL_Image.open(io.BytesIO(edited_image_bytes)))

In [ ]:
# --- 3단계: 아웃페인팅 ---
print("\n--- 🚀 3단계: 편집된 이미지 16:9 비율로 아웃페인팅 ---")

initial_image_pil = PIL_Image.open(edited_image_path)
mask_pil = PIL_Image.new("L", initial_image_pil.size, 0)

image_height = 500
image_width = int(image_height * 16 / 9)
target_size = (image_width, image_height)

image_pil_outpaint, mask_pil_outpaint = pad_image_and_mask(
    initial_image_pil, mask_pil, target_size, 0, 0
)

image_pil_outpaint_image = Image(image_bytes=get_bytes_from_pil(image_pil_outpaint))
mask_pil_outpaint_image = Image(image_bytes=get_bytes_from_pil(mask_pil_outpaint))

raw_ref_image = RawReferenceImage(reference_image=image_pil_outpaint_image, reference_id=0)
mask_ref_image = MaskReferenceImage(
    reference_id=1,
    reference_image=mask_pil_outpaint_image,
    config=MaskReferenceConfig(mask_mode="MASK_MODE_USER_PROVIDED", mask_dilation=0.03),
)

outpaint_prompt = """
This image is a crop of a much larger photograph.
Reveal the rest of the original scene.
Extend the environment naturally, maintaining the exact same lighting, mood, and fine details as the provided image.
"""
outpainted_result = CLIENT.models.edit_image(
    model="imagen-3.0-capability-001",
    prompt=outpaint_prompt,
    reference_images=[raw_ref_image, mask_ref_image],
    config=EditImageConfig(
        edit_mode="EDIT_MODE_OUTPAINT", number_of_images=1,
        safety_filter_level="BLOCK_MEDIUM_AND_ABOVE", person_generation="ALLOW_ADULT"
    ),
)

generated_image_bytes = outpainted_result.generated_images[0].image.image_bytes
output_filename = os.path.join(OUTPUT_DIR, "sample_imgs", "edited_image_result.png")

with open(output_filename, "wb") as f:
    f.write(generated_image_bytes)
print(f"✅ 아웃페인팅 이미지가 '{output_filename}' 파일로 저장되었습니다.")

print("\n--- ✅ 아웃페인팅 결과 이미지 ---")
display(PIL_Image.open(io.BytesIO(generated_image_bytes)))

In [12]:
input_img_file = "veo_story_telling/sample_imgs/edited_image_result.png"
veo_prompt = "사진속 두 모델이 자연스러운 포즈를 취하는 영상"

vid_result = generate_video_from_image(input_img_file, veo_prompt, 1)

✅ VeoImage 객체가 성공적으로 생성되었습니다!
비디오 생성 완료를 기다리는 중...
비디오 생성 완료를 기다리는 중...
비디오 생성 완료를 기다리는 중...
비디오 생성 완료를 기다리는 중...
비디오 생성 완료를 기다리는 중...
비디오 생성 완료를 기다리는 중...
'veo_story_telling/scene_1_video.mp4' 파일로 비디오를 성공적으로 저장했습니다.


In [ ]:
# input_img_file = "veo_story_telling/sample_imgs/edited_image_result.png"
# veo_prompt = "화장품을 광고하기 위해 팔에 문지르는 영상"

# vid_result = generate_video_from_image(input_img_file, veo_prompt, 1)